# ***TP2 - MLVOT***

In [1]:
import pandas as pd

## **1. Loading and checking the best data**

In [27]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [28]:
def load_data(path, sep=","):
    return pd.read_csv(path, sep=sep, names= ['frame', 'id', 'bb_left', 'bb_top', 'bb_width',
'bb_height', 'conf', 'x', 'y', 'z'])

In [29]:
def check_data(df):
    print(bcolors.HEADER + "General info :" + bcolors.ENDC)
    print(df.info())

    print(bcolors.HEADER + "\n\nStats :" + bcolors.ENDC)
    print(df.describe())

    print(bcolors.HEADER + "\n\nNull entries :" + bcolors.ENDC)
    print(df.isnull().sum())

    print(bcolors.HEADER + "\n\nTypes :" + bcolors.ENDC)
    print(df.dtypes)

### *Public dataset*

In [32]:
path_det = "files/det/public-dataset/det.txt"
df_det_public = load_data(path_det)
df_det_public['conf'] /= 100
df_det_public

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1689,385,146.620,332.710,0.67567,-1,-1,-1
1,1,-1,1303,503,61.514,139.590,0.29439,-1,-1,-1
2,1,-1,1258,569,40.123,91.049,0.19601,-1,-1,-1
3,1,-1,31,525,113.370,257.270,0.17013,-1,-1,-1
4,1,-1,1800,483,94.660,214.810,0.11949,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
4897,525,-1,1479,580,28.302,64.224,0.36629,-1,-1,-1
4898,525,-1,1806,521,36.723,83.333,0.23327,-1,-1,-1
4899,525,-1,1820,381,123.420,280.060,0.14567,-1,-1,-1
4900,525,-1,1655,425,94.660,214.810,0.12803,-1,-1,-1


In [33]:
check_data(df_det_public)

General info :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4902 entries, 0 to 4901
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   frame      4902 non-null   int64  
 1   id         4902 non-null   int64  
 2   bb_left    4902 non-null   int64  
 3   bb_top     4902 non-null   int64  
 4   bb_width   4902 non-null   float64
 5   bb_height  4902 non-null   float64
 6   conf       4902 non-null   float64
 7   x          4902 non-null   int64  
 8   y          4902 non-null   int64  
 9   z          4902 non-null   int64  
dtypes: float64(3), int64(7)
memory usage: 383.1 KB
None


Stats :
             frame      id      bb_left       bb_top     bb_width  \
count  4902.000000  4902.0  4902.000000  4902.000000  4902.000000   
mean    273.949612    -1.0  1266.308650   432.289678   120.214085   
std     144.119813     0.0   522.326834    94.557726    64.348312   
min       1.000000    -1.0   -52.000000     1.000000   

### *YOLOv5s*

In [30]:
path_det = "files/det/Yolov5l/det.txt"
df_det_s = load_data(path_det, sep=" ")
df_det_s

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1700,391,156,337,0.914550,-1,-1,-1
1,1,-1,250,456,107,248,0.883148,-1,-1,-1
2,1,-1,1255,539,60,118,0.826354,-1,-1,-1
3,1,-1,1288,459,73,199,0.745969,-1,-1,-1
4,1,-1,120,504,93,239,0.740778,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
4557,525,-1,1432,457,96,193,0.841939,-1,-1,-1
4558,525,-1,1340,439,95,218,0.815440,-1,-1,-1
4559,525,-1,119,503,87,244,0.639956,-1,-1,-1
4560,525,-1,1676,440,58,201,0.568499,-1,-1,-1


In [31]:
check_data(df_det_s)

General info :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4562 entries, 0 to 4561
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   frame      4562 non-null   int64  
 1   id         4562 non-null   int64  
 2   bb_left    4562 non-null   int64  
 3   bb_top     4562 non-null   int64  
 4   bb_width   4562 non-null   int64  
 5   bb_height  4562 non-null   int64  
 6   conf       4562 non-null   float64
 7   x          4562 non-null   int64  
 8   y          4562 non-null   int64  
 9   z          4562 non-null   int64  
dtypes: float64(1), int64(9)
memory usage: 356.5 KB
None


Stats :
             frame      id      bb_left       bb_top     bb_width  \
count  4562.000000  4562.0  4562.000000  4562.000000  4562.000000   
mean    275.546909    -1.0  1125.018413   398.295923   124.803376   
std     147.465487     0.0   590.615114    98.967614    83.902927   
min       1.000000    -1.0     0.000000     0.000000   

### *YOLOv5l*

In [34]:
path_det = "files/det/Yolov5l/det.txt"
df_det_l = load_data(path_det, sep=" ")
df_det_l

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1700,391,156,337,0.914550,-1,-1,-1
1,1,-1,250,456,107,248,0.883148,-1,-1,-1
2,1,-1,1255,539,60,118,0.826354,-1,-1,-1
3,1,-1,1288,459,73,199,0.745969,-1,-1,-1
4,1,-1,120,504,93,239,0.740778,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
4557,525,-1,1432,457,96,193,0.841939,-1,-1,-1
4558,525,-1,1340,439,95,218,0.815440,-1,-1,-1
4559,525,-1,119,503,87,244,0.639956,-1,-1,-1
4560,525,-1,1676,440,58,201,0.568499,-1,-1,-1


In [35]:
check_data(df_det_l)

General info :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4562 entries, 0 to 4561
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   frame      4562 non-null   int64  
 1   id         4562 non-null   int64  
 2   bb_left    4562 non-null   int64  
 3   bb_top     4562 non-null   int64  
 4   bb_width   4562 non-null   int64  
 5   bb_height  4562 non-null   int64  
 6   conf       4562 non-null   float64
 7   x          4562 non-null   int64  
 8   y          4562 non-null   int64  
 9   z          4562 non-null   int64  
dtypes: float64(1), int64(9)
memory usage: 356.5 KB
None


Stats :
             frame      id      bb_left       bb_top     bb_width  \
count  4562.000000  4562.0  4562.000000  4562.000000  4562.000000   
mean    275.546909    -1.0  1125.018413   398.295923   124.803376   
std     147.465487     0.0   590.615114    98.967614    83.902927   
min       1.000000    -1.0     0.000000     0.000000   

### *Best choice overall*

I will be keeping YOLOv5s as the best choice for the detection data, as it has the best overall performance, similar to YOLOv5l but with a smaller model size.